In [ ]:
!conda install ephem --yes

In [49]:
import ephem
import pandas
from SciServer import SciDrive

In [2]:
#SciDrive.directoryList('mpc')
mpc = SciDrive.download('mpc/MPCORB.DAT', localFilePath='mpcorb.dat')
print('ok')

ok


In [3]:
df = pandas.read_fwf('mpcorb.dat')
df = df.set_index('desig')
df = df.rename(columns = {'epc':'epoch', 'meanAn': 'meanAnomaly', 'perihel':'periheleonArgument'})
df = df.rename(columns = {'nodAL':'nodeAscendingLongitude', 'incl': 'inclination'})
df = df.rename(columns = {'motDail': 'motionDailyMean', 'z':'uncertainty', 'ref': 'reference'})
df = df.rename(columns = {'nOb': 'nObservations', 'nP':'nOppositions', 'arcL':'arcLength'})
df = df.rename(columns = {'rms':'rmsResidual', 'p': 'perturbersCoarse', 'pp':'perturbersPrecise'})
df = df.rename(columns = {'comp':'computerName', 'flg':'flags', 'lastO':'lastObservationDate'})
df['lastObservationDate'] = df['lastObservationDate'].apply(lambda x: str(x))
print('ok')

ok


In [5]:
df['lastObservationDate'] = pandas.to_datetime(df['lastObservationDate'].apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:8]), errors='coerce')
print(df['lastObservationDate'])

desig
 00001   2017-03-05
 00002   2017-06-17
 00003   2017-06-15
 00004   2017-03-23
 00005   2017-06-15
 00006   2017-05-28
 00007   2017-06-28
 00008   2017-07-04
 00009   2017-06-23
 00010   2017-06-27
 00011   2017-03-05
 00012   2017-06-29
 00013   2017-05-17
 00014   2017-06-23
 00015   2017-06-19
 00016   2017-06-18
 00017   2016-12-25
 00018   2017-04-10
 00019   2016-12-22
 00020   2016-11-23
 00021   2017-05-21
 00022   2017-06-15
 00023   2017-07-02
 00024   2017-07-03
 00025   2017-06-18
 00026   2017-06-23
 00027   2017-06-29
 00028   2017-06-25
 00029   2017-06-18
 00030   2017-07-04
            ...    
 K17P2   2017-08-05
 K17P2   2017-08-11
 K17P2   2017-08-11
 K17P2   2017-08-12
 K17P2   2017-08-12
 K17P2   2017-08-14
 K17P2   2017-08-14
 K17P2   2017-08-14
 K17P2   2017-08-15
 K17P2   2017-08-14
 K17P2   2017-08-16
 PLS21   1960-09-29
 PLS41   1960-09-28
 PLS45   1960-09-28
 PLS47   1960-09-28
 PLS48   1960-09-28
 PLS60   1960-09-28
 PLS68   1960-09-28
 T2S10   1973-

In [56]:

# names, {f|e|h|p|E|P}, incli, long-ascending-node, arg-of-perihel, a (mean dist), 
##  mean-daily-motion, eccentricity, mean-anomaly, epoch, equinox-year, mag1, mag2, ang-size

'''
line = "C/2002 Y1 (Juels-Holvorcem),
e,
103.7816,
166.2194,
128.8232,
242.5695,
0.0002609,
0.99705756,
0.0000,
04/13.2508/2003,
2000,
g  
6.5,
4.0"
'''
#df['orbit_type'] = 'e'

#print(line)
xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly', 'epoch', 'epoch', 'h', 'g']

xephem_df = df[xephem_column_order]
#print(xephem_df.head(1))
#K01W68W 16.1   0.15 K1794 347.98048  138.34564  218.49218    8.98681  0.1019559  0.17780077   3.1321817  0 E2017-QB4   101   9 2001-2017 0.39 M-v 38h MPCLINUX   0400          2001 WW68          20170824
#yh = ephem.readdb(xephem_df.iloc[0].astype(str))
#yh.compute('2007/10/1')
#print('{:.1f}'.format(yh.earth_distance))
#print(yh.mag)
#df.columns

with open('orbits.csv', 'w') as f:
    for i in range(0,3):
        thisline = ''
        for j in range(0,2):
            thisline += "'" + xephem_df.iloc[i].values[j] + "'" + ", "
        for k in range(3,12):
            thisline += str(xephem_df.iloc[i].values[k]) + ", "
        thisline += str(xephem_df.iloc[i].values[12])
        thisline += '\n'
        f.write(thisline)
#    thisline += '\n'
#    for j in range(0, len(xephem_df.iloc[i].values)):
#        print(xephem_df.iloc[i].values[j])
print('Done')

Done


In [ ]:
>>> line = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
>>> yh = ephem.readdb(line)
>>> yh.compute('2007/10/1')
>>> print('%.10f' % yh.earth_distance)
14.8046731949
>>> print(yh.mag)
23.96